### ⚙️ Flags de Controle de Execução

Estas variáveis controlam a execução de blocos específicos do notebook:

- `RESET_DATABASES = False`:  
  Define se o banco de dados deve ser resetado e os dados recarregados a partir dos arquivos CSV originais.  
  > `True` = Apaga e recria as tabelas, recarregando os dados.  
  > `False` = Mantém os dados existentes no banco.

- `ADD_ENTRIES = False`:  
  Define se novas entradas (arquivos `.csv` na pasta `entries_folder`) devem ser enviadas para o endpoint `/tag_financas`.  
  > `True` = Envia arquivos e move os processados para `old_entries_folder`.  
  > `False` = Ignora essa etapa.

> ✅ Altere essas flags conforme necessário para controlar o comportamento do notebook sem modificar a lógica interna.

In [1]:
RESET_DATABASES = False
ADD_ENTRIES = False

In [2]:

from database.finance_schema import FinanceEntrySchema, AccountSchema
from database import FinanceDB
from tqdm import tqdm
from time import time
import pandas as pd
import requests, os, shutil

In [3]:
user_id = "d0db18d3-b0a5-4e0b-8456-e1ca560cd574"
url = "http://127.0.0.1:3000"
body = {"user_id":user_id}

In [4]:
data_folder = 'data'
os.makedirs(data_folder, exist_ok=True)

backup_folder = 'data/backup'
os.makedirs(backup_folder, exist_ok=True)

entries_folder = "entries"
os.makedirs(entries_folder, exist_ok=True)

old_entries_folder = 'entries/processed'
os.makedirs(old_entries_folder, exist_ok=True)

In [38]:
data = requests.post(f"{url}/financas", json=body).json()

In [39]:
financas = [FinanceEntrySchema(**entry) for entry in data["financas"]]
financas_df = pd.DataFrame(data["financas"])

accounts = [AccountSchema(**acc) for acc in data["accounts"]]
accounts_df = pd.DataFrame(data["accounts"])

In [40]:
financas_df.to_csv(f"{backup_folder}/financas_{time()}.csv", index=False, encoding="utf-8-sig")
accounts_df.to_csv(f"{backup_folder}/accounts_{time()}.csv", index=False, encoding="utf-8-sig")

In [41]:
response = requests.post(f'{url}/resume_financas', json=body|data).json()
print(f"Saldo Total: R${response['saldo_total']:.2f}")
for acc in response['saldo_por_conta']:
    print(f"Saldo {acc['nome']}: R${acc['valor']:.2f}")

Saldo Total: R$1.19
Saldo Conta Corrente XP: R$820.18
Saldo Cartão de Crédito XP: R$-818.99


In [42]:
news = financas_df[financas_df["need_tagging"] == True]

In [43]:
news

,id,data,valor,destino_origem,descricao,tipo,categoria,subcategoria,nome,conta,notas,need_tagging
463,464,2025-06-14T12:24:51,-45.92,CAFE CARINO GOIANIA BR,CAFE CARINO GOIANIA BR,Gasto,,,,1,,True
464,465,2025-06-14T17:04:36,-37.12,Ingresso.Com Ltda,Pix enviado para Ingresso.Com Ltda,Gasto,,,,1,,True
465,466,2025-06-19T20:40:32,-20.49,Nuvei do Brasil Instituicao de Pagamento Ltda.,Pix enviado para Nuvei do Brasil Instituicao d...,Gasto,,,,1,,True
466,467,2025-06-21T22:05:06,-23.99,Okto Tech Ltda.,Pix enviado para Okto Tech Ltda.,Gasto,,,,1,,True
467,468,2025-07-05T11:55:10,-182.47,Nuvei do Brasil Instituicao de Pagamento Ltda.,Pix enviado para Nuvei do Brasil Instituicao d...,Gasto,,,,1,,True
468,469,2025-07-20T14:42:00,-23.99,Nuvei do Brasil Ltda.,Pix enviado para Nuvei do Brasil Ltda.,Gasto,,,,1,,True
469,470,2025-07-15T00:00:00,1527.06,Pagamentos Validos Normais,Compra no Cartão de Crédito,Transferência,,,,2,,True
470,471,2025-07-19T00:00:00,-14.97,UBER * PENDING,Compra no Cartão de Crédito,Gasto,,,,2,,True


### 🔄 Restauração do Banco de Dados

Esta célula é responsável por **resetar as tabelas do banco de dados** e **recarregar os dados a partir de arquivos CSV** (`financas.csv` e `accounts.csv`).  
Ela só será executada se a flag `RESET_DATABASES` estiver ativada (`True`).

- Conecta ao banco via `FinanceDB`.
- Executa `reset_tables()` para limpar e recriar as tabelas.
- Carrega os dados:
  - **Finanças**: de `financas.csv`, convertendo cada linha em `FinanceEntrySchema`, garantindo que o campo `notas` esteja presente (mesmo que vazio).
  - **Contas**: de `accounts.csv`, convertendo cada linha em `AccountSchema`.
- Em caso de erro, imprime o erro ocorrido.
- Garante que o banco será fechado no final da execução com `db.close()`.

> ⚠️ Use com cuidado: essa operação sobrescreve os dados atuais do banco.

In [11]:
def load_csv_and_insert(path, insert_fn, build_schema_fn, fillna_dict=None):
    if not os.path.exists(path):
        return
    
    df = pd.read_csv(path)
    if fillna_dict:
        df.fillna(fillna_dict, inplace=True)
    
    with tqdm(total=len(df), desc=f"Inserting from {os.path.basename(path)}") as pbar:
        for _, row in df.iterrows():
            schema = build_schema_fn(row)
            insert_fn(schema)     
            pbar.update(1)

In [ ]:
RESET_DATABASES = True
if RESET_DATABASES:

    db = FinanceDB("d0db18d3-b0a5-4e0b-8456-e1ca560cd574")
    try:
        db.reset_tables()
        print('loading financas')
        load_csv_and_insert(
            f"{data_folder}/financas.csv",
            db.insert_financa,
            lambda row: FinanceEntrySchema(**{**row, "notas": row.get("notas", "") or ""}),
            {"notas": ""}
        )
        print('loading accounts')
        load_csv_and_insert(
            f"{data_folder}/accounts.csv",
            db.insert_account,
            lambda row: AccountSchema(
                nome=row.get("nome", ""),
                saldo_inicial=row.get("saldo_inicial", 0.0),
                open_finance_id=row.get("open_finance_id"),
            ),
        )
    except Exception as e:
        print(f"Erro geral ao restaurar: {e}")

    finally:
        db.close()

loading financas


Inserting from financas.csv: 100%|██████████| 463/463 [03:09<00:00,  2.45it/s]


loading accounts


Inserting from accounts.csv: 100%|██████████| 2/2 [00:00<00:00,  2.44it/s]


### 📤 Envio de Novas Entradas para Processamento

Esta célula envia arquivos CSV com novas entradas financeiras para a API `/tag_financas`.  
A execução ocorre apenas se a flag `ADD_ENTRIES` estiver ativada (`True`).

#### Etapas:
1. Percorre todos os arquivos da pasta `entries_folder`.
2. Filtra apenas arquivos com extensão `.csv`.
3. Para cada arquivo CSV:
   - Envia o conteúdo via `POST` para o endpoint de tagueamento financeiro.
   - Lança erro caso a requisição falhe (`raise_for_status()`).
   - Move o arquivo para a pasta `old_entries_folder` após o envio com sucesso, para evitar reprocessamentos futuros.

> 💡 Útil para processar novos dados financeiros de forma automática, garantindo que não sejam enviados novamente no futuro.


In [37]:
ADD_ENTRIES = True
if ADD_ENTRIES:
    for file in os.listdir(entries_folder):
        if not file.endswith('.csv'): continue
        file_path = fr"{entries_folder}/{file}"
        with open(file_path, "rb") as f:
            response = requests.post(f"{url}/tag_financas", files={"file": f})
            response.raise_for_status()
        shutil.move(file_path, os.path.join(old_entries_folder, os.path.basename(file_path)))